## Embedding на мед. данных 

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
# легковесная встраиваемая векторная бд
from langchain_community.vectorstores import Chroma

# модель для эмбеддингов, при первом запуске скачается в ~/.cache/huggingface/hub
# https://huggingface.co/sergeyzh/LaBSE-ru-turbo
embeddings = HuggingFaceEmbeddings(
            model_name= "sergeyzh/LaBSE-ru-turbo",
            model_kwargs={'device': 'cpu'}
        )

# embeddings.embed_documents(["Зеленый слоник", "покушать принес"])

c:\Users\Очир\Desktop\Учёба\RAG_LLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cardiology_tests = [
    {
        "query": "Какие целевые значения артериального давления при гипертонии?",
        "relevant_chunk": """
        Гипертоническая болезнь - хроническое заболевание, характеризующееся стойким повышением 
        артериального давления выше 140/90 мм рт.ст. Целевое АД для большинства пациентов: 
        < 140/90 мм рт.ст. Для пациентов с сахарным диабетом и хронической болезнью почек 
        целевой уровень < 130/80 мм рт.ст.
        """,
        "expected_answer_contains": ["< 140/90 мм рт.ст", "целевое АД", "для большинства пациентов"]
    },
    {
        "query": "Препараты первой линии для лечения гипертонической болезни",
        "relevant_chunk": """
        Основные классы антигипертензивных препаратов: ингибиторы АПФ (эналаприл, лизиноприл, рамиприл), 
        блокаторы кальциевых каналов (амлодипин, нифедипин), бета-блокаторы (бисопролол, метопролол), 
        диуретики (гидрохлоротиазид, индапамид). Препараты первой линии: ингибиторы АПФ и блокаторы 
        кальциевых каналов.
        """,
        "expected_answer_contains": ["ингибиторы АПФ", "блокаторы кальциевых каналов", "эналаприл", "амлодипин"]
    }
]

endocrinology_tests = [
    {
        "query": "Диагностические критерии сахарного диабета 2 типа",
        "relevant_chunk": """
        Сахарный диабет 2 типа диагностируется при: уровне глюкозы плазмы натощак ≥ 7.0 ммоль/л, 
        глюкозе плазмы через 2 часа после ПГТТ ≥ 11.1 ммоль/л, гликированном гемоглобине (HbA1c) ≥ 6.5%. 
        Для подтверждения требуется дважды проведенный анализ в разные дни.
        """,
        "expected_answer_contains": ["глюкоза натощак ≥ 7.0 ммоль/л", "HbA1c ≥ 6.5%", "ПГТТ ≥ 11.1 ммоль/л"]
    },
    {
        "query": "Какая начальная доза метформина при диабете?",
        "relevant_chunk": """
        Метформин - препарат первой линии при сахарном диабете 2 типа. Начальная доза 500-850 мг 1-2 раза в сутки. 
        Максимальная суточная доза 2000-2550 мг. Принимается во время или после еды для уменьшения 
        gastrointestinal побочных эффектов.
        """,
        "expected_answer_contains": ["500-850 мг", "начальная доза", "во время или после еды"]
    }
]

emergency_medicine_tests = [
    {
        "query": "Неотложная терапия при остром коронарном синдроме",
        "relevant_chunk": """
        При остром коронарном синдроме неотложная терапия включает: аспирин 300 мг разжевать, 
        клопидогрел 300-600 мг нагрузочная доза, нефракционированный гепарин или низкомолекулярный гепарин. 
        При боли - морфин 2-4 мг в/в. Кислородотерапия при сатурации < 90%.
        """,
        "expected_answer_contains": ["аспирин 300 мг", "клопидогрел", "гепарин", "морфин"]
    },
    {
        "query": "В чем разница между ИМпST и ИМбпST?",
        "relevant_chunk": """
        Инфаркт миокарда с подъемом сегмента ST (ИМпST) характеризуется подъемом сегмента ST на ЭКГ 
        и требует экстренной реперфузии (ЧКВ или тромболизис). Инфаркт миокарда без подъема сегмента ST 
        (ИМбпST) проявляется депрессией сегмента ST или инверсией зубца Т, тактика - медикаментозная терапия 
        и плановая коронарография.
        """,
        "expected_answer_contains": ["подъем сегмента ST", "экстренная реперфузия", "депрессия сегмента ST", "медикаментозная терапия"]
    }
]

complex_tests = [
    {
        "query": "Ведение пациента с гипертонией и сердечной недостаточностью",
        "relevant_chunk": [
            """
            При гипертонической болезни целевое АД < 140/90 мм рт.ст. Препараты выбора: 
            ингибиторы АПФ, блокаторы кальциевых каналов, диуретики.
            """,
            """
            При хронической сердечной недостаточности основная терапия: ИАПФ/БРА, бета-блокаторы, 
            антагонисты альдостерона. При сочетании с гипертонией предпочтение отдается ИАПФ и бета-блокаторам.
            """
        ],
        "expected_answer_contains": ["ИАПФ", "бета-блокаторы", "целевое АД < 140/90", "хроническая сердечная недостаточность"]
    },
    {
        "query": "Дифференциальная диагностика болей в груди",
        "relevant_chunk": [
            """
            Острый коронарный синдром: давящая боль за грудиной, иррадиирует в левую руку, шею, челюсть, 
            сопровождается одышкой, потливостью. Длительность от 20 минут.
            """,
            """
            Перикардит: острая боль, усиливается при кашле, глубоком дыхании, в положении лежа, 
            уменьшается при наклоне вперед. Шум трения перикарда при аускультации.
            """,
            """
            ТЭЛА: острая боль, сочетается с одышкой, тахикардией, может быть кровохарканье. 
            Факторы риска: иммобилизация, онкология, тромбофилия.
            """
        ],
        "expected_answer_contains": ["острый коронарный синдром", "перикардит", "ТЭЛА", "дифференциальная диагностика"]
    }
]

In [3]:

combed = cardiology_tests + endocrinology_tests + emergency_medicine_tests + complex_tests
texts = []
metadata = []
query = []
for el in combed:
    query.append(el.get("query"))
    texts.append(el.get("relevant_chunk") if isinstance(el.get("relevant_chunk"), str) else " ".join(el.get("relevant_chunk")))
    metadata.append({"source": "doc-1", "page": 1})

vectorstore = Chroma(
    collection_name="medical_docs",
    embedding_function=embeddings,
    persist_directory="./chroma_db"
)


C:\Users\Очир\AppData\Local\Temp\ipykernel_12332\2480548705.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [4]:
vectorstore.add_texts(texts, metadata)

['21d15002-661a-4a6d-b6f9-656f55a3eb26',
 'fde8eece-6ce0-4283-8803-562d00e3d4df',
 '1e33c39f-acb8-4c5a-bcea-b8acc1ded84c',
 'f16f987b-900e-4dda-9530-9386d5d74612',
 '39014058-9917-4001-8c21-977dd7900415',
 '4e3d7075-54a1-4cea-8bde-f9dc5fde16b5',
 '9ef05e4c-b6c9-4d4b-b5aa-47b9cf86eb78',
 '32a73630-2a33-4b35-a447-ef120d8a1421']

In [5]:
from IPython.display import display

In [6]:
i = 1
print(q := query[i])
display(ans := vectorstore.similarity_search_with_relevance_scores(query=query[i], k=2))

Препараты первой линии для лечения гипертонической болезни


[(Document(metadata={'source': 'doc-1', 'page': 1}, page_content='\n            При гипертонической болезни целевое АД < 140/90 мм рт.ст. Препараты выбора: \n            ингибиторы АПФ, блокаторы кальциевых каналов, диуретики.\n             \n            При хронической сердечной недостаточности основная терапия: ИАПФ/БРА, бета-блокаторы, \n            антагонисты альдостерона. При сочетании с гипертонией предпочтение отдается ИАПФ и бета-блокаторам.\n            '),
  0.6844239544881345),
 (Document(metadata={'page': 1, 'source': 'doc-1'}, page_content='\n        Основные классы антигипертензивных препаратов: ингибиторы АПФ (эналаприл, лизиноприл, рамиприл), \n        блокаторы кальциевых каналов (амлодипин, нифедипин), бета-блокаторы (бисопролол, метопролол), \n        диуретики (гидрохлоротиазид, индапамид). Препараты первой линии: ингибиторы АПФ и блокаторы \n        кальциевых каналов.\n        '),
  0.6399000237212261)]

In [7]:
ans = [el[0].page_content for el in ans]
ans

['\n            При гипертонической болезни целевое АД < 140/90 мм рт.ст. Препараты выбора: \n            ингибиторы АПФ, блокаторы кальциевых каналов, диуретики.\n             \n            При хронической сердечной недостаточности основная терапия: ИАПФ/БРА, бета-блокаторы, \n            антагонисты альдостерона. При сочетании с гипертонией предпочтение отдается ИАПФ и бета-блокаторам.\n            ',
 '\n        Основные классы антигипертензивных препаратов: ингибиторы АПФ (эналаприл, лизиноприл, рамиприл), \n        блокаторы кальциевых каналов (амлодипин, нифедипин), бета-блокаторы (бисопролол, метопролол), \n        диуретики (гидрохлоротиазид, индапамид). Препараты первой линии: ингибиторы АПФ и блокаторы \n        кальциевых каналов.\n        ']

### cross-encoder для re-rank

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# https://huggingface.co/ncbi/MedCPT-Cross-Encoder
tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Cross-Encoder")
model = AutoModelForSequenceClassification.from_pretrained("ncbi/MedCPT-Cross-Encoder")



# combine query article into pairs
pairs = [[q, article] for article in ans]

with torch.no_grad():
    encoded = tokenizer(
        pairs,
        truncation=True,
        padding=True,
        return_tensors="pt",
        max_length=512,
    )

    logits = model(**encoded).logits.squeeze(dim=1)
    probabilities = F.softmax(logits, dim=0)
    
    print(logits)
    print(probabilities)

tensor([15.7108, 15.8161])
tensor([0.4737, 0.5263])


In [15]:
vectorstore.delete_collection()